## Семинар 1 Индекс

## Intro

### работа с файлами и папками

In [ ]:
import os

curr_dir = os.getcwd()
filepath = os.path.join(curr_dir, 'test.txt')

### os.path  
путь до файла

In [ ]:
# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath(filepath))


# возвращает имя файла / папки по полному ти до него
print(os.path.basename(filepath))


# проверить существование директории - True / False
print(os.path.exists(curr_dir))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
os.listdir(curr_dir)

При обходе файлов не забывайте исключать системные директории, такие как .DS_Store

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
for root, dirs, files in os.walk(curr_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(curr_dir))

### чтение файла 

In [ ]:
fpath = 'test.txt'


# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

    
#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

    
#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 

Напоминание про enumerate:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, не надо так

### CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
 
# инициализируем
vectorizer = CountVectorizer()

# составляем корпус документов
corpus = [
  'слово1 слово2 слово3',
  'слово2 слово3',
  'слово1 слово2 слово1',
  'слово4'
]

# считаем
X = vectorizer.fit_transform(corpus)
print(X)

# получится следующая структура:
#        | слово1 | слово2 | слово3 | слово4
# текст1 |   1    |    1   |   1    |   0
# текст2 |   0    |    1   |   1    |   0
# текст3 |   2    |    1   |   0    |   0
# текст4 |   0    |    0   |   0    |   1
 
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer
# порядок совпадает с матрицей
print(vectorizer.get_feature_names())  # ['слово1', 'слово2', 'слово3', 'слово4']
 
# чтобы узнать индекс токена в словаре
print(vectorizer.vocabulary_.get('слово3')) # вернет 2
 
# показать матрицу
X.toarray()
 
# теперь можно быстро подсчитать вектор для нового документа
print(vectorizer.transform(["слово1 слово4 слово4"]))  # результат [[1 0 0 2]]
 
# чтобы узнать количественное вхождение каждого слова:
matrix_freq = np.asarray(X.sum(axis=0)).ravel()
print(matrix_freq)
final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])
print(final_matrix)

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзей. Одна серия - один документ.

**To do:** 

**1 Создайте индекс этой базы в формате json и в формате матрицы Term-Document**

Компоненты вашей реализации:
    - функция препроцессинга данных
    - функция индексирования данных

**2 С помощью обратного индекса в формате Term-Document посчитайте:** 


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) кто из главных героев статистически самый популярный? 



[download_friends_corpus](https://yadi.sk/d/4wmU7R8JL-k_RA?w=1)

Напоминание про defaultdict: 
> Если ваш код предполагает использование multiple values словаря, рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

> ```from collections import defaultdict
d = defaultdict(list)
d['example'].append('example1')
d['example'].append('example2')```

In [ ]:
### _check : в коллекции должно быть около 165 файлов

In [74]:
import numpy as np
import re
import os
import pymorphy2
import pickle
import json

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

## Функция препроцессинга

In [75]:
morphy_norm = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words('russian')

def my_preprocessor(text):
    text = text.lower()
    text = re.sub(r'[\'\:\;\»\«\>\—]"”\,\!\?\.\-\(\)\[\]', ' ', str(text).rstrip("']"))
    text = text.replace('\ufeff', '')
    words = [i for i in text.split() if i not in stop_words]
    words = [morphy_norm.parse(i)[0].normal_form for i in words]
    return " ".join(words)

## Создадим обратный индекс

In [76]:
serial_dir = 'friends-data'
documents_array = []
file_names = []

for root, dirs, files in os.walk(serial_dir):
    for name in files:
        file_names.append(os.path.join(root, name))
            
file_names = sorted(file_names)
for i in file_names:
    with open(i, 'r') as f:  
        documents_array.append(f.read())
file_names = [i.split('/')[-1] for i in file_names]

vectorizer = CountVectorizer(preprocessor=my_preprocessor)
count_vector = vectorizer.fit_transform( documents_array )

X = vectorizer.fit_transform(documents_array)
X.toarray()

matrix_freq = np.asarray(X.sum(axis=0)).ravel()
final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])
vec_of_words = vectorizer.get_feature_names()

zipped_word_freq = [(int(i[0]), i[1]) for i in list( zip(final_matrix[1], final_matrix[0]) )]
len(zipped_word_freq)

25197

## Преобразуем в json вида { 'слово': [ в 1 документе, во 2 документе, ...], ...}

In [77]:
json_dict = dict()
for word in vec_of_words:
    json_dict[word] = []

for series_ind, vec in enumerate(X):
    for word_ind, num in enumerate(vec.toarray()[0]):
#         if num != 0:
#             json_dict[vec_of_words[word_ind]].append( (file_names[series_ind], num) )
        json_dict[vec_of_words[word_ind]].append( int(num) )
len(json_dict) # количество слов

25197

## a) 10 самых частовстречаемых слов

In [78]:
matrix_freq = np.asarray(X.sum(axis=0)).ravel()
final_matrix = np.array([np.array(vectorizer.get_feature_names()), matrix_freq])

zipped_word_freq = [(int(i[0]), i[1]) for i in list( zip(final_matrix[1], final_matrix[0]) )]
zipped_word_freq = sorted(zipped_word_freq, key=lambda x: int(x[0]))

In [79]:
zipped_word_freq[-10:][::-1]

[(6519, 'это'),
 (2537, 'что'),
 (2479, 'нет'),
 (2298, 'да'),
 (2112, 'ты'),
 (1715, 'мочь'),
 (1584, 'то'),
 (1560, 'хотеть'),
 (1459, 'мой'),
 (1268, 'весь')]

## b) 10 первых слов, встречающиеся по 1 разу

In [80]:
counter = 10
for i in zipped_word_freq:
    if i[0] > 1 or counter == 0:
        break
    else:
        counter -= 1
        print(i)

(1, '03815')
(1, '0fps')
(1, '101')
(1, '102')
(1, '110')
(1, '1100')
(1, '112')
(1, '11б')
(1, '1250')
(1, '128')


## c) Набор слов, которые есть во всех документах коллекции

In [81]:
for k, vec in json_dict.items():
    if np.count_nonzero(vec) == len(vec):
        print(k)

быть
да
мой
нет
просто
то
ты
хотеть
что
это


## d) Самый популярный персонаж

In [82]:
persons = ["рэйчел", "чендлер", "фиби", "джоуя", "росс", "моника"]
name_freq = []
for freq, name in zipped_word_freq:
    for i in persons:
        if name == i:
            name_freq.append((freq, name))
sorted(name_freq, key=lambda x: x[0])[::-1]

[(856, 'росс'),
 (575, 'фиби'),
 (559, 'моника'),
 (386, 'чендлер'),
 (237, 'рэйчел'),
 (222, 'джоуя')]

## Трансформация в pickle

In [83]:
# словарь в json
with open("output.json", "w") as f:
    json.dump(json_dict, f)

# словать в пикл
with open('output_dict.pickle', 'wb') as f:
    pickle.dump(json_dict, f)
    
#матрицу в пикл
with open('output_mtx.pickle', 'wb') as f:
    pickle.dump(X, f) 